In [1]:
import numpy as np
import pickle
import pandas as pd
import cv2
import json
import os
import matplotlib.pyplot as plt
#from utils.map_util import Razmetka
#rm=Razmetka()

In [2]:
import xml.etree.ElementTree as ET

def transform_points(pstr):
    return np.array([[float(p1) for p1 in b.split(',')] for b in pstr.split(';')])

# Откройте XML файл
with open("./annotations/mx4.xml", "r") as f:
    tree = ET.parse(f)

root = tree.getroot()
plist=[]
# Итерируйте по дереву
for child in root:
    print(child.tag)
    if child.tag=='image':
        for child1 in child:
            #print(child1.attrib['label'],child1.attrib['points'])
            if 'points' in child1.attrib.keys():
                plist.append({'label':child1.attrib['label'],
                            'points':transform_points(child1.attrib['points'])})

version
meta
image


In [3]:
pidx=[]
for idx,p in enumerate(plist):
    if p['label']=='one_point':
        pidx.append(idx)
        
zlist=[plist[i] for i in pidx]

In [4]:
def sort_points(a):
    # располангает длинную сторону по горизонтали сортирует точки tl,tr,br,bl
    if (a[:,0].max()-a[:,0].min())<(a[:,1].max()-a[:,1].min()):
        long_axis=1
        short_axis=0
    else:
        long_axis=0
        short_axis=1
        
    #find left,right
    left=a[a[:,long_axis]<a[:,long_axis].mean()]
    right=a[a[:,long_axis]>a[:,long_axis].mean()]
    tl=left[left[:,short_axis]==left[:,short_axis].min()][0]
    bl=left[left[:,short_axis]==left[:,short_axis].max()][0]
    tr=right[right[:,short_axis]==right[:,short_axis].min()][0]
    br=right[right[:,short_axis]==right[:,short_axis].max()][0]

    return np.array([tl,tr,br,bl],dtype=np.float32)

In [4]:
def sort_points(a):
    # располагает длинную сторону по горизонтали сортирует точки tl,tr,br,bl
    long_axis = np.argmax(np.max(a, axis=0) - np.min(a, axis=0))
    short_axis = 1 - long_axis

    # find left,right
    left = a[a[:, long_axis] < np.mean(a[:, long_axis])]
    right = a[a[:, long_axis] > np.mean(a[:, long_axis])]
    tl = left[left[:, short_axis] == np.min(left[:, short_axis])][0]
    bl = left[left[:, short_axis] == np.max(left[:, short_axis])][0]
    tr = right[right[:, short_axis] == np.min(right[:, short_axis])][0]
    br = right[right[:, short_axis] == np.max(right[:, short_axis])][0]

    return np.array([tl, tr, br, bl], dtype=np.float32)

In [5]:
def get_chessboard_nodes(rows, cols, cell_size):
  """
  Возвращает точки пересечения клеток шахматной доски.

  Args:
    rows: Количество клеток в строках.
    cols: Количество клеток в столбцах.
    cell_size: Размер клетки.

  Returns:
    Множество точек пересечения клеток.
  """

  nodes = set()
  for row in range(rows):
    for col in range(cols):
      nodes.add((row * cell_size, col * cell_size))
      nodes.add((row * cell_size + cell_size, col * cell_size))
      nodes.add((row * cell_size, col * cell_size + cell_size))
      nodes.add((row * cell_size + cell_size, col * cell_size + cell_size))

  return nodes

In [6]:
def generate_internal_nodes(n_rows, n_cols, field_size):
  """
  Генерирует положение внутренних узлов шахматной доски.

  Args:
    n_rows: Количество строк.
    n_cols: Количество столбцов.
    field_size: Размер поля.

  Returns:
    Список положений внутренних узлов. значения x,y
  """
  pos=[]
  nodes = []
  for row in range(1, n_rows):
    for col in range(1, n_cols ):
      nodes.append((col * field_size,row * field_size, ))
      pos.append([col,row])
  return np.array(nodes,dtype=np.float32),np.array(pos,dtype=int)

In [7]:
def get_start_and_end_points(nodes, position):
  """
  Возвращает начальную и конечную точки столбца узлов.

  Args:
    nodes: Список положений узлов.
    position: Массив с координатами узлов.

  Returns:
    Начальная и конечная точки столбца узлов.
  """

  # Уникальные значения первого столбца массива position.
  unique_cols = np.unique(position[:, 0])
  unique_rows = np.unique(position[:, 1])

  # Начальные и конечные точки столбцов узлов.
  column_lines = [
      (nodes[position[:, 0] == col][position[position[:, 0] == col][:,1].argmin()],
       nodes[position[:, 0] == col][position[position[:, 0] == col][:,1].argmax()])
      for col in unique_cols
  ]

  # Начальные и конечные точки строк узлов.
  row_lines = [
      (nodes[position[:, 1] == row][position[position[:, 1] == row][:,0].argmin()],
       nodes[position[:, 1] == row][position[position[:, 1] == row][:,0].argmax()])
      for row in unique_rows
  ]

  return np.array(column_lines,dtype=int), np.array(row_lines,dtype=int)

In [8]:
import skimage as ski
def get_nodes_box(nodes,order=['tl','tr','br','bl']):
    res={}
    res['tl']=[0,0]
    res['tr']=[nodes[:,0].max()+nodes[1,0]-nodes[0,0],0]
    res['br']=[res['tr'][0],nodes[:,1].max()+nodes[1,0]-nodes[0,0]]
    res['bl']=[res['tl'][0],res['br'][1]]
    return np.array([res[pos] for pos in order],dtype=np.float32)

nodes,position=generate_internal_nodes(6,12,20)    
obj_box=get_nodes_box(nodes)
img_pts=[]
for ob in zlist:
    img_box=sort_points(ob['points'].copy())
    tform = ski.transform.ProjectiveTransform()
    tform.estimate(obj_box,img_box)
    image_points=tform(nodes)
    corners=np.expand_dims(image_points,1)
    img_pts.append(corners.astype(np.float32))
    cl,rl=get_start_and_end_points(image_points, position)

In [25]:
mx1=cv2.imread('./images/undistorted/v1.png')
mx2=cv2.imread('./images/original/511.bmp')

In [12]:
objp=np.zeros((nodes.shape[0],3),dtype=np.float32)
objp[:,:2]=nodes[:,:2].copy()
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera([objp for i in range(len(img_pts))], img_pts, (mx1.shape[1],mx1.shape[0]), None, None)
h,  w = mx1.shape[:2]
newcameramtx, roi = cv2.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))
dst = cv2.undistort(mx1, mtx, dist, None, newcameramtx)